In [ ]:
!pip install pinecone langchain

In [ ]:
#setting api keys and the file path

In [6]:
pinecone_api_key = PINECONE_API_KEY
cohere_api_key = COHERE_API_KEY
folder_path = FOLDER_PATH
index_name = PINECONE_INDEX_NAME

In [ ]:
# setting up the embeddings anf the pinecone vector base

In [8]:
!pip install langchain-cohere
!pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.5
    Uninstalling aiohttp-3.10.5:
      Successfully uni

In [ ]:
# getting the index from pinecone and if the index is not ther making one instead and then getting it

In [26]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
Indexs=[]
for index in pc.list_indexes():
    name = index.name
    Indexs.append(f'{name}')

# making the index if not already there
if index_name not in Indexs:
    pc.create_index(name=index_name,dimension=1024)

index = pc.Index(index_name)

In [ ]:
# could have used chroma or FIASS vector store as well

In [27]:
from langchain_cohere import CohereEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore


embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key,model="embed-english-v3.0")
vectorstore = PineconeVectorStore(index = index, pinecone_api_key = pinecone_api_key,embedding = embeddings)

In [28]:
pc.describe_index(index_name)

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'sample-set-px57zr3.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'sample-set',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [ ]:
# write now we dont have any vectors stored in the index so it is showing the count as 0.

In [37]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [30]:
!pip install PyPdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [31]:
from PyPDF2 import PdfReader
def pdf_to_text(file):
    """Convert the PDF file to text."""
    pdf_reader = PdfReader(file)  # Creates a PDF reader object
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()  # Extracts text from each page
    return text

def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file given its path."""
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [33]:
# making a function to update the vectorstore

In [32]:
!pip install langchain_community langchain_core

In [44]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


def update_vectorstore_with_files(file=None,vectorstore=vectorstore):
    """Update the vector store with the PDFs uploaded by the user"""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    if file:
        text = pdf_to_text(file)
        text_chunks = text_splitter.split_text(text)
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        if not documents:
            return
        vectorstore.add_documents(documents=documents)
    else:
        text_loader_kwargs = {"autodetect_encoding": True}
        loader = DirectoryLoader(path = folder_path,loader_cls=TextLoader,loader_kwargs=text_loader_kwargs)
        directory = loader.load()
        documents = []
        for file in directory:
            file_path = file.metadata.get('source')
            text=extract_text_from_pdf(file_path)
            splits = text_splitter.split_text(text)  # Splits text into chunks
            text_chunks = [Document(page_content=chunk) for chunk in splits]
            documents.extend(text_chunks)

        vectorstore.add_documents(documents=documents)

In [45]:
update_vectorstore_with_files(vectorstore=vectorstore)

In [ ]:
# now we added the vectors to the store so we can see we have 8 vectors in the vectorstore now.

In [46]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 8}},
 'total_vector_count': 8}

In [ ]:
#initializing our model and the prompt we are going to use

In [47]:
from langchain_cohere import ChatCohere


llm = ChatCohere(model='command-r',cohere_api_key=cohere_api_key)

In [48]:
from langchain.prompts import PromptTemplate

template ="""You are a helpful AI assistant. Use the following context to answer the user's question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
        you will be given data related to company and u will answer the questions using those details
        Context: {context}
        Query: {input}
        Answer:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
# initializing the cohere re ranker

In [50]:
from langchain_cohere import CohereRerank

reranker = CohereRerank(model='rerank-english-v3.0',cohere_api_key = cohere_api_key)

In [ ]:
# now initializing the retriever

In [51]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
            base_retriever=vectorstore.as_retriever(),
            base_compressor = reranker)

In [ ]:
# Making langchain chains we are using the create_retrieval_chain

In [53]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(
            llm = llm,
            prompt = prompt)

In [54]:
from langchain.chains.retrieval import create_retrieval_chain

chain = create_retrieval_chain(
            retriever = compression_retriever,
            combine_docs_chain = question_answer_chain
        )

In [59]:
# now the chain is ready to use so we will ask it a simple query!!

In [60]:
query = "what is the company name?"

In [58]:
response = chain.invoke({"input": query})

In [61]:
answer = response["answer"]
print(answer)

The company name is Innovate Solutions Inc.


In [62]:
query2 = "name some projects of the company realted to AI"

In [64]:
response2 = chain.invoke({"input": query2})

In [65]:
answer = response2["answer"]
print(answer)

The company has worked on at least two projects involving AI:

- Project 2: Healthcare AI Assistant - Created an AI-powered virtual assistant for healthcare professionals.
- Project 3: E-commerce Personalization - Implemented a recommendation engine using machine learning algorithms for an e-commerce platform.

The company also mentions winning an "Innovation Award" for its work in AI and machine learning, and appears to specialize in AI-based solutions, so there may be further AI-related projects not explicitly mentioned in the context provided.


In [ ]:
#now we have asked some questions lets update the vector store and see what happens

In [66]:
new_file_path="new_projects.pdf"

In [68]:
with open(new_file_path, 'rb') as file:
  update_vectorstore_with_files(file=file,vectorstore=vectorstore)

In [ ]:
# lets check if the file got uploaded properly or not

In [69]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11}},
 'total_vector_count': 11}

In [ ]:
#as the total vector count increased that means the vector store was updated with the new files

In [70]:
# lets check the chain with the same query as the last time and see the response

In [71]:
response3 = chain.invoke({"input": query2})

In [72]:
answer = response3["answer"]
print(answer)

Based on the provided information, here are some projects related to AI:

1. Healthcare AI Assistant: This project aims to create an AI-powered virtual assistant for healthcare professionals, improving efficiency and providing personalized recommendations.

2. Autonomous Vehicle Navigation: The project focuses on developing an advanced navigation system for self-driving vehicles, which requires real-time AI processing for mapping, obstacle detection, and path planning.

3. Natural Language Processing Suite: This suite uses AI for text analysis and sentiment analysis, benefiting customer feedback analysis and content moderation.

4. Energy Efficiency Optimization: The project involves the development of an energy management system, utilizing AI and IoT sensors for real-time monitoring and control to optimize energy consumption in commercial buildings.

5. Personalized Health Monitoring: HealthTech Innovations is working on a wearable device with AI capabilities to offer personalized hea

In [ ]:
# as we can see the number of projects increased as we had details about a few new projects in the new file we uploaded